In [31]:
%matplotlib notebook
from astropy.cosmology import LambdaCDM
import numpy as np
import matplotlib.pyplot as plt
from astropy import constants as const
import astropy.units as u
from scipy.integrate import quad

In [32]:
cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7, Tcmb0=2.725)

In [44]:
z = np.arange(0, 1.5, 0.01)
z=0.4

In [45]:
phi_star = 3.6 * cosmo.efunc(z)**2
alpha = -1.05 * (1 + z)**(-2/3)
fr = 0.8*(1 + z)**(-1/2)

In [35]:
alpha

array([-1.05      , -1.04305782, -1.03622926, -1.02951137, -1.02290132,
       -1.01639636, -1.00999383, -1.00369119, -0.99748597, -0.99137577,
       -0.98535829, -0.9794313 , -0.97359264, -0.96784023, -0.96217204,
       -0.95658611, -0.95108056, -0.94565355, -0.9403033 , -0.93502809,
       -0.92982625, -0.92469616, -0.91963624, -0.91464499, -0.90972091,
       -0.90486257, -0.90006858, -0.89533759, -0.89066828, -0.88605938,
       -0.88150964, -0.87701786, -0.87258286, -0.86820351, -0.8638787 ,
       -0.85960735, -0.85538841, -0.85122085, -0.84710369, -0.84303596,
       -0.83901671, -0.83504502, -0.83112001, -0.82724079, -0.82340653,
       -0.81961638, -0.81586956, -0.81216526, -0.80850273, -0.80488121,
       -0.80129997, -0.79775831, -0.79425552, -0.79079093, -0.78736388,
       -0.78397371, -0.78061981, -0.77730154, -0.77401832, -0.77076955,
       -0.76755466, -0.76437308, -0.76122426, -0.75810768, -0.7550228 ,
       -0.75196911, -0.74894612, -0.74595332, -0.74299024, -0.74

Need to compute the cluster volume...

$M_{vir} = 4/3 \pi r^3_{vir} \rho_c(r<r_{vir}) = 4/3 \pi r^3_{vir} \Delta_c \rho_c$

if we let $\Delta_c = 200$ then 

$M_{200} = 4/3 \pi r^3_{200} 200 \rho_c$ with $\rho_c = \frac{3H(z)^2}{8\pi G}$

or just $M_{200} = V_{200}200\rho_c$

Don't forget that $H(z) = H_0E(z)$

In [48]:
def rho_crit(z, cosmo):
    # convert G into better units:
    G = const.G.to(u.km**2 * u.Mpc/(u.M_sun * u.s**2))
    return 3 / (8 * np.pi * G) * cosmo.H0**2 * cosmo.efunc(z)**2 # Mpc^3

# So now we are going to calculate the volumes as a function of z

M200 = 1e15 * u.solMass
V200 = M200/ (200 * rho_crit(z, cosmo))

In [49]:
V200

<Quantity 24.13520299 Mpc3>

The Schechter Function:

For Luminosity:

$\Phi(L) = \phi^\star \frac{L}{L_\star}^\alpha e^{-\frac{L}{L_\star}}$

For Magnitudes:

$\Phi(M) = \phi^\star\frac{2}{5}log(10) (10^{\frac{2}{5}(M_\star - M)})^{\alpha+1} e^{-10^{\frac{2}{5}(M_\star - M)}}$



In [38]:
def schechterL(luminosity, phiStar, alpha, LStar): 
    """Schechter luminosity function.""" 
    LOverLStar = (luminosity/LStar) 
    return (phiStar/LStar) * LOverLStar**alpha * np.exp(- LOverLStar) 

def schechterM(magnitude, phiStar, alpha, MStar): 
    """Schechter luminosity function by magnitudes.""" 
    MStarMinM = 0.4 * (MStar - magnitude) 
    try:
        return (0.4 * np.log(10) * phiStar * 10.0**(MStarMinM * (alpha + 1.)) * np.exp(-10.**MStarMinM))
    except OverflowError:
        return 0.0

In [39]:
M200 = 10 # 10^14 Msun
Mpiv = 6 # 10^14
zpiv = 0.6

alpha = -0.96 * (M200 / Mpiv)**0.01 * ((1 + z)/ (1 + zpiv))**-0.94
Phi = 1.68 * (M200 / Mpiv)**0.09 * ((1 + z)/ (1 + zpiv))**0.09 * cosmo.efunc(z)**2
fr = 0.62 * (M200 / Mpiv)**0.08 * ((1 + z)/ (1 + zpiv))** -0.80

In [51]:
M_star = -20.44 + 5 * np.log10(0.7) # abs mag.
M_star = -21.946521706412092 # abs mag @ z= 0.4
M_star_sub = M_star - 2.5 * np.log10(0.4)
M_star_big = M_star - 2.5 * np.log(100)
#M_star = 20.071802735723015 # app mag @ z= 0.4
y, err = quad(schechterM, M_star_big, -18.01828907152557, args=(phi_star, alpha, M_star))

In [52]:
(y * V200.value + 1) * fr

208.04202985039478

In [41]:
Phi

array([1.68618364, 1.70303894, 1.72021516, 1.73771604, 1.75554533,
       1.77370677, 1.7922041 , 1.81104106, 1.83022139, 1.84974883,
       1.86962711, 1.88985998, 1.91045117, 1.9314044 , 1.95272342,
       1.97441196, 1.99647375, 2.01891253, 2.04173203, 2.06493598,
       2.08852811, 2.11251216, 2.13689187, 2.16167095, 2.18685316,
       2.21244221, 2.23844186, 2.26485582, 2.29168783, 2.31894164,
       2.34662098, 2.37472958, 2.40327118, 2.43224952, 2.46166833,
       2.49153137, 2.52184236, 2.55260505, 2.58382319, 2.6155005 ,
       2.64764075, 2.68024766, 2.713325  , 2.7468765 , 2.78090591,
       2.81541698, 2.85041346, 2.88589911, 2.92187766, 2.95835289,
       2.99532853, 3.03280835, 3.0707961 , 3.10929555, 3.14831044,
       3.18784454, 3.2279016 , 3.26848541, 3.3095997 , 3.35124826,
       3.39343485, 3.43616323, 3.47943717, 3.52326044, 3.56763682,
       3.61257008, 3.65806399, 3.70412233, 3.75074886, 3.79794738,
       3.84572166, 3.89407548, 3.94301262, 3.99253687, 4.04265

In [42]:
phi_star

array([ 3.6       ,  3.63273137,  3.66611754,  3.70016499,  3.73488021,
        3.77026968,  3.8063399 ,  3.84309735,  3.88054852,  3.9186999 ,
        3.95755798,  3.99712924,  4.03742017,  4.07843727,  4.12018701,
        4.16267589,  4.2059104 ,  4.24989702,  4.29464224,  4.34015255,
        4.38643445,  4.43349441,  4.48133893,  4.5299745 ,  4.5794076 ,
        4.62964473,  4.68069237,  4.73255702,  4.78524515,  4.83876327,
        4.89311787,  4.94831542,  5.00436242,  5.06126536,  5.11903074,
        5.17766503,  5.23717473,  5.29756634,  5.35884633,  5.4210212 ,
        5.48409745,  5.54808155,  5.61298   ,  5.6787993 ,  5.74554593,
        5.81322637,  5.88184714,  5.9514147 ,  6.02193556,  6.09341621,
        6.16586313,  6.23928282,  6.31368177,  6.38906646,  6.4654434 ,
        6.54281907,  6.62119996,  6.70059257,  6.78100338,  6.8624389 ,
        6.9449056 ,  7.02840998,  7.11295854,  7.19855776,  7.28521414,
        7.37293417,  7.46172433,  7.55159114,  7.64254106,  7.73

In [13]:
1/5

0.2

Here is the stuff from the c++ code.

In [60]:
def dAintegrand2(y, alpha, mstar):
    #val = y**alpha * np.exp(-y)
    # using mags instead of luminosity
    val = 10.0**(-2 / 5 * (y - mstar) * (alpha + 1.)) * np.exp(-10.** (-2 / 5) * (y - mstar))
    return val
    
def calc_hon(z, mlimit, mstar):
    M200 = 10 # 10^14 Msun
    Mpiv = 6 # 10^14
    zpiv = 0.6
    
    alpha = -0.96 * (M200 / Mpiv)**0.01 * ((1 + z)/ (1 + zpiv))**-0.94
    Phi = 1.68 * (M200 / Mpiv)**0.09 * ((1 + z)/ (1 + zpiv))**0.09 * cosmo.efunc(z)**2
    fr = 0.62 * (M200 / Mpiv)**0.08 * ((1 + z)/ (1 + zpiv))** -0.80
    
    
    R200 = M200 * 4301.8644383/ (cosmo.H0.value**2 * cosmo.efunc(z)**2)
    R200 = R200**(1 / 3)
    
    # Calculate factors
    Vol = 4 / 3 * np.pi * R200**3 # This gives basically the same number as my calculation above.
    # Using Mags Instead of Luminosity
    NN = -0.4 * np.log(10) * Vol * Phi * quad(dAintegrand2, 0.0, mlimit, args=(alpha, mstar))[0]
    print(NN)
    return (NN + 1) * fr

In [95]:
calc_hon(z, -21, M_star)

140.48553340043802


101.68225839692074

In [53]:
type(np.log(10))

numpy.float64

In [38]:
cosmo.efunc(z)**2

1.523360401525354

In [80]:
from scipy import special

In [97]:
phi_star * special.gammainc(alpha + 1, 3) * V200 * fr

<Quantity 89.24134572 Mpc3>

In [82]:
alpha

-1.093961783729938